# Part 3 - Summarization

For Part 3, use the same 300-word long sections as in Part 1, but instead of answering the questions about these sections, ask the ChatPI to summarize them for you, using Hugging Face summarization pipeline.

Part 3.2 -- use two different HF summarization models: use the default summarization pipeline, then use other models of choice and discuss the differences in the result.

https://huggingface.co/docs/transformers/main_classes/pipelines

https://huggingface.co/docs/transformers/v4.35.0/en/main_classes/pipelines#transformers.SummarizationPipeline


In [ ]:
!pip3 install -r ../requirements.txt

%load_ext autoreload
%autoreload 2


In [ ]:
from collections import defaultdict

from src import utils
from src.summarization import run, run_models
import evaluate
# from datasets import load_metric

---
---

## Experiments & Results

Use the same 300-word long sections as in Part 1, but instead of answering the questions about these sections, ask the ChatPI to summarize them for you, using Hugging Face summarization pipeline.


In [ ]:
# TODO: Try out a good selection of models and keep some interesting ones
models = [
    "sshleifer/distilbart-cnn-12-6",
    "slauw87/bart_summarisation",
    "pszemraj/pegasus-x-large-book-summary",
]


In [ ]:
"""
Rouge1: unigram (1-gram) based scoring
Rouge2: bigram (2-gram) based scoring
RougeL: ROUGE-L measures the longest common subsequence between the generated summary and the reference summary.
RougeLsum: splits text using \n
"""

ctx = "../sections/resolution.0.md"
text = ""
rouge = evaluate.load("rouge")

with open(ctx, "r") as f:
    text = f.read().strip()


for model in models:
    sum_text = run(text, model=model)
    rouge = evaluate.load("rouge")
    results = rouge.compute(
        predictions=[sum_text],
        references=[text],
        use_stemmer=True,
    )
    print("EVAL RESULTS FOR MODEL: ", model)
    print(results)